<a href="https://colab.research.google.com/github/psantul/Mtech_thesis_2022/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 198 kB 52.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=70d954838d5976ea08a45c1a3daa744a6448ef3de6a8432d693185f70090e1ab
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
pip install findspark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier

from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [5]:
spark = SparkSession.builder.appName("Classification with Spark").getOrCreate()

In [6]:
dataset = spark.read.csv('/content/drive/MyDrive/LALR.csv',inferSchema=True, header =True)

In [7]:
dataset.count()

499998

In [8]:
dataset = dataset.filter(dataset.modelMag_u > 0)
dataset = dataset.filter(dataset.modelMag_g > 0)
dataset = dataset.filter(dataset.modelMag_r > 0)
dataset = dataset.filter(dataset.modelMag_i > 0)
dataset = dataset.filter(dataset.modelMag_z > 0)
dataset = dataset.filter(dataset.modelMag_ug > 0)
dataset = dataset.filter(dataset.modelMag_gr > 0)
dataset = dataset.filter(dataset.modelMag_ri > 0)
dataset = dataset.filter(dataset.modelMag_iz > 0)
dataset = dataset.filter(dataset.fiberMag_u > 0)
dataset = dataset.filter(dataset.fiberMag_g > 0)
dataset = dataset.filter(dataset.fiberMag_r > 0)
dataset = dataset.filter(dataset.fiberMag_i > 0)
dataset = dataset.filter(dataset.fiberMag_z > 0)
dataset = dataset.filter(dataset.fiberMag_ug > 0)
dataset = dataset.filter(dataset.fiberMag_gr > 0)
dataset = dataset.filter(dataset.fiberMag_ri > 0)
dataset = dataset.filter(dataset.fiberMag_iz > 0)
dataset = dataset.filter(dataset.petroR50_rr > 0)
dataset = dataset.filter(dataset.petroR90_zz > 0)
dataset = dataset.filter(dataset.ri > 0)
dataset = dataset.filter(dataset.iz > 0)
dataset = dataset.filter(dataset.dered_u > 0)
dataset = dataset.filter(dataset.dered_g > 0)
dataset = dataset.filter(dataset.dered_r > 0)
dataset = dataset.filter(dataset.dered_i > 0)
dataset = dataset.filter(dataset.dered_z > 0)
dataset = dataset.filter(dataset.petroMag_uu > 0)
dataset = dataset.filter(dataset.petroMag_gg > 0)
dataset = dataset.filter(dataset.petroMag_rr > 0)
dataset = dataset.filter(dataset.petroMag_ii > 0)
dataset = dataset.filter(dataset.petroMag_zz > 0)

In [9]:
dataset.count()

403041

In [10]:
dataset = dataset.distinct()

In [11]:
dataset.count()

403041

In [12]:
import pyspark.sql.functions as func
dataset = dataset.withColumn("redshift", func.round(dataset["redshift"], 2).cast('integer'))

In [13]:
dataset.count()

403041

In [14]:
dataset.show()

+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-------+-------+--------+--------+--------+--------+--------+-----------+-----------+-----------+-----------+-----------+--------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|modelMag_ug|modelMag_gr|modelMag_ri|modelMag_iz|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|fiberMag_ug|fiberMag_gr|fiberMag_ri|fiberMag_iz|petroR50_rr|petroR90_zz|     ri|     iz| dered_u| dered_g| dered_r| dered_i| dered_z|petroMag_uu|petroMag_gg|petroMag_rr|petroMag_ii|petroMag_zz|redshift|
+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-------+-------+--------+--------+--------+--

In [15]:
dataset.printSchema()

root
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- modelMag_ug: double (nullable = true)
 |-- modelMag_gr: double (nullable = true)
 |-- modelMag_ri: double (nullable = true)
 |-- modelMag_iz: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- fiberMag_ug: double (nullable = true)
 |-- fiberMag_gr: double (nullable = true)
 |-- fiberMag_ri: double (nullable = true)
 |-- fiberMag_iz: double (nullable = true)
 |-- petroR50_rr: double (nullable = true)
 |-- petroR90_zz: double (nullable = true)
 |-- ri: double (nullable = true)
 |-- iz: double (nullable = true)
 |-- dered_u: double (nullable = true)
 |-- dered_g: double (nullable = true)

In [16]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [17]:
new_data.printSchema()

root
 |-- modelMag_u: float (nullable = true)
 |-- modelMag_g: float (nullable = true)
 |-- modelMag_r: float (nullable = true)
 |-- modelMag_i: float (nullable = true)
 |-- modelMag_z: float (nullable = true)
 |-- modelMag_ug: float (nullable = true)
 |-- modelMag_gr: float (nullable = true)
 |-- modelMag_ri: float (nullable = true)
 |-- modelMag_iz: float (nullable = true)
 |-- fiberMag_u: float (nullable = true)
 |-- fiberMag_g: float (nullable = true)
 |-- fiberMag_r: float (nullable = true)
 |-- fiberMag_i: float (nullable = true)
 |-- fiberMag_z: float (nullable = true)
 |-- fiberMag_ug: float (nullable = true)
 |-- fiberMag_gr: float (nullable = true)
 |-- fiberMag_ri: float (nullable = true)
 |-- fiberMag_iz: float (nullable = true)
 |-- petroR50_rr: float (nullable = true)
 |-- petroR90_zz: float (nullable = true)
 |-- ri: float (nullable = true)
 |-- iz: float (nullable = true)
 |-- dered_u: float (nullable = true)
 |-- dered_g: float (nullable = true)
 |-- dered_r: float (nu

In [18]:
from pyspark.sql.functions import col, count, isnan, when
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+---+---+-------+-------+-------+-------+-------+-----------+-----------+-----------+-----------+-----------+--------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|modelMag_ug|modelMag_gr|modelMag_ri|modelMag_iz|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|fiberMag_ug|fiberMag_gr|fiberMag_ri|fiberMag_iz|petroR50_rr|petroR90_zz| ri| iz|dered_u|dered_g|dered_r|dered_i|dered_z|petroMag_uu|petroMag_gg|petroMag_rr|petroMag_ii|petroMag_zz|redshift|
+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+---+---+-------+-------+-------+-------+-------+-----------+-----------

In [23]:
cols=new_data.columns
# cols = ['modelMag_u', 'modelMag_g', 'modelMag_r']
# cols.remo/ve("Classes")
cols.remove("redshift")
assembler = VectorAssembler(inputCols=cols,outputCol="features")

# Now let us use the transform method to transform our dataset
data=assembler.transform(new_data)

data.select("features",'redshift').show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|features                                                                                                                                                                                                                                                                                                                                                                                

In [24]:
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
data=standardscaler.fit(data).transform(data)

In [26]:
data.select("features",'redshift','Scaled_features').show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
assembled_data = data.select("Scaled_features","redshift")
assembled_data.show()

+--------------------+--------+
|     Scaled_features|redshift|
+--------------------+--------+
|[11.1425068956343...|     0.0|
|[9.38896887839148...|     0.0|
|[9.26234681785694...|     0.0|
|[8.76688574397992...|     2.0|
|[9.94997939826152...|     0.0|
|[9.08303427808620...|     0.0|
|[9.76236394483268...|     0.0|
|[9.51257809117957...|     0.0|
|[8.96426971154025...|     0.0|
|[7.95035469325975...|     0.0|
|[8.95467419161798...|     0.0|
|[10.7902187394855...|     0.0|
|[11.3154493682620...|     0.0|
|[9.47381930447416...|     0.0|
|[8.39195636441214...|     0.0|
|[10.5849303916556...|     0.0|
|[8.72147638082965...|     0.0|
|[11.0177028910648...|     0.0|
|[9.96518591260506...|     0.0|
|[9.16433670611169...|     0.0|
+--------------------+--------+
only showing top 20 rows



In [28]:
train, test = assembled_data.randomSplit([0.7, 0.3])

In [29]:
train.show()

+--------------------+--------+
|     Scaled_features|redshift|
+--------------------+--------+
|[5.81304195702285...|     0.0|
|[5.82268436322613...|     0.0|
|[5.97506679744247...|     0.0|
|[5.98182650574829...|     0.0|
|[5.99522385635249...|     0.0|
|[6.03133801429415...|     0.0|
|[6.04909215130282...|     0.0|
|[6.06933813246990...|     0.0|
|[6.09434832565568...|     0.0|
|[6.10264679320093...|     0.0|
|[6.1279152651243,...|     0.0|
|[6.13349715771598...|     0.0|
|[6.1537985131977,...|     0.0|
|[6.15634128555789...|     0.0|
|[6.16725891776600...|     0.0|
|[6.17095040401074...|     0.0|
|[6.18792040838886...|     0.0|
|[6.21448593268925...|     0.0|
|[6.21610068003927...|     0.0|
|[6.24193704183167...|     0.0|
+--------------------+--------+
only showing top 20 rows



In [30]:
test.show()

+--------------------+--------+
|     Scaled_features|redshift|
+--------------------+--------+
|[5.96230079486349...|     0.0|
|[5.99273605411485...|     0.0|
|[6.11943854887283...|     0.0|
|[6.12033706786134...|     0.0|
|[6.15163689396668...|     0.0|
|[6.18072417263923...|     0.0|
|[6.24632818456274...|     0.0|
|[6.28109436192560...|     0.0|
|[6.29530130926123...|     0.0|
|[6.30663000481226...|     0.0|
|[6.34481685972786...|     0.0|
|[6.37388271622030...|     0.0|
|[6.46188584050828...|     0.0|
|[6.46802309301459...|     0.0|
|[6.46822235970885...|     0.0|
|[6.46987510111416...|     0.0|
|[6.47818650280589...|     0.0|
|[6.48965949492862...|     0.0|
|[6.48982036337551...|     0.0|
|[6.50806599795054...|     0.0|
+--------------------+--------+
only showing top 20 rows



In [31]:
naive_bayes = NaiveBayes(featuresCol='Scaled_features',labelCol='redshift',smoothing=1.0)

In [32]:
model = naive_bayes.fit(train)

In [33]:
# select example rows to display.
prediction_test = model.transform(test)

In [34]:
prediction_test.show()

+--------------------+--------+--------------------+--------------------+----------+
|     Scaled_features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[5.96230079486349...|     0.0|[-618.94244750797...|[0.98893843613215...|       0.0|
|[5.99273605411485...|     0.0|[-592.73256943396...|[0.99843594946643...|       0.0|
|[6.11943854887283...|     0.0|[-597.61851005676...|[0.91113675505743...|       0.0|
|[6.12033706786134...|     0.0|[-603.65975645288...|[0.99808113743590...|       0.0|
|[6.15163689396668...|     0.0|[-600.94089642329...|[0.58908904792086...|       0.0|
|[6.18072417263923...|     0.0|[-606.48606520840...|[0.65749824358878...|       0.0|
|[6.24632818456274...|     0.0|[-621.29442096412...|[0.98520064940280...|       0.0|
|[6.28109436192560...|     0.0|[-644.84333643636...|[0.99125241668367...|       0.0|
|[6.29530130926123...|     0.0|[-618.84942707553...|[0.9975841062

In [35]:
prediction_test.select("redshift","prediction").show(10)

+--------+----------+
|redshift|prediction|
+--------+----------+
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
+--------+----------+
only showing top 10 rows



In [36]:
predictionAndLabels = prediction_test.select("redshift","prediction").rdd

In [37]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="redshift", predictionCol="prediction", metricName="accuracy")
accuracy_NB = evaluator.evaluate(prediction_test)

In [38]:
print ("Accuracy",accuracy_NB)

Accuracy 0.9272689887732862


In [39]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Area under ROC = 0.7825246722042294


In [40]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="redshift", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(labelCol="redshift", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(prediction_test)
print("Mean Absolute Eroor (MAE) on test data = %g" % mae)



Root Mean Squared Error (RMSE) on test data = 0.466874
Mean Absolute Eroor (MAE) on test data = 0.107793


In [41]:
from pyspark.mllib.evaluation import MulticlassMetrics

# Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Overall statistics
precision = metrics.precision(0.0)
recall = metrics.recall(0.0)
f1Score = metrics.fMeasure(0.0)
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Summary Stats
Precision = 0.9530600646467516
Recall = 0.9849675122058662
F1 Score = 0.9687511282810413
